<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Natural_Language_Processing/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import torch
import torchvision
import torch.nn as nn #All N.N. modules, nn.Linear, nn.Conv2d, Batch Norm, Loss Functions
import torch.optim as optim #SGD, Adam etc
import torch.nn.functional as F #All functions that doesnt have any parameters
from torch.utils.data import DataLoader #Give easier dataset management and creates mini batches
import torchvision.datasets as datasets #Standard Datasets
import torchvision.transforms as transforms

#Hyperparameters

In [9]:
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 256
learning_rate = 0.001
batch_size = 64
num_epochs = 2

#Create RNN

In [3]:
#batch, channel, 28 , 28
#IN RNN -> Consider as 28 sequences with 28 features
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True) #batch should be the first dimension
        self.fc = nn.Linear( hidden_size*sequence_length, num_classes )
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        #Forward Propogation
        out, _ = self.rnn(x, h0) #output, hidden_state
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out


#Set Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data

In [ ]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#Initialize Network

In [10]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

#Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Train Network

In [12]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device).squeeze(1)  #batch x 1 x 28 x 28 --> RNN need batch x 28 x 28 i.e. sequeeze axis 1
        targets = targets.to(device)

        #forward
        scores = model(data)
        loss = criterion(scores, targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients

#Check Accuracy

In [14]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).squeeze(1) #RNN Changes from NN
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1) #RNN desired input unlike NN

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 57648/ 60000 with accuracy 96.08 

Checking Accuracy on Test Data
Got 9590/ 10000 with accuracy 95.90 

